# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.u

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/326db5c901e2681584ec5c06fc17f6c96e516ff9/Al.

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355147705846                   -1.10    6.0    160ms
  2   -9.356676729567       -2.82       -1.43    1.0    108ms
  3   -9.356963798455       -3.54       -2.83    2.4    146ms
  4   -9.357011422599       -4.32       -3.02    6.6    471ms
  5   -9.357011849360       -6.37       -3.14    1.0   78.7ms
  6   -9.357012160588       -6.51       -3.28    2.6    103ms
  7   -9.357012373358       -6.67       -3.43    1.0   79.2ms
  8   -9.357012477287       -6.98       -3.60    1.0    103ms
  9   -9.357012529763       -7.28       -3.87    1.0   78.2ms
 10   -9.357012547199       -7.76       -4.14    3.5    117ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.74295648033                   -0.97    6.4    518ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     1
│     1
│     1
│     7
│     1
│     7
│     1
│     1
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  2   -18.73126754850   +   -1.93       -1.38    2.5    255ms
  3   -18.79196279264       -1.22       -1.72    4.9    346ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│      1
│     12
│      1
│      1
│     10
│      1
│      1
│      1
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  4   -18.79227014772       -3.51       -1.98    3.5    322ms
  5   -18.79238783944       -3.93       -2.61    1.8    260ms
  6   -18.79240643059       -4.73       -3.07    5.9    439ms
  7   -18.79240750993       -5.97       -3.22    2.9    340ms
  8   -18.7

In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.53950063564                   -0.84   10.0    1.55s
  2   -37.55581402496       -1.79       -1.20    2.4    1.06s
  3   -37.56005001657       -2.37       -2.27    6.0    1.17s
  4   -37.56446395279       -2.36       -2.26   17.2    1.85s
  5   -37.56450995725       -4.34       -2.41    2.4    860ms
  6   -37.56455235123       -4.37       -3.50    1.1    1.06s
  7   -37.56455321307       -6.06       -3.75   13.9    1.95s
  8   -37.56455403272       -6.09       -4.24    2.2    850ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355173413526                   -1.10    5.9    168ms
  2   -9.356722702058       -2.81       -1.90    1.0   64.1ms
  3   -9.356951420499       -3.64       -2.38    6.5    153ms
  4   -9.356999139006       -4.32       -2.67    1.6   71.1ms
  5   -9.357012154953       -4.89       -3.67    1.0   65.3ms
  6   -9.357012492436       -6.47       -3.90    8.5    179ms
  7   -9.357012537924       -7.34       -4.26    1.4   71.9ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54576191541                   -0.84    8.1    1.80s
  2   -37.53620484430   +   -2.02       -1.47    3.5    755ms
  3   -22.98722645648   +    1.16       -0.47    9.9    1.78s
  4   -30.65129521785        0.88       -0.63    9.1    2.20s
  5   -37.38847478752        0.83       -1.35    6.2    1.38s
  6   -34.52789122307   +    0.46       -0.83    6.2    1.27s
  7   -37.49451762961        0.47       -1.55    6.6    1.32s
  8   -37.55131024668       -1.25       -1.79    2.0    1.11s
  9   -37.55494948803       -2.44       -2.00    1.4    724ms
 10   -37.56320999304       -2.08       -2.32    2.6    967ms
 11   -37.56418994899       -3.01       -2.52    3.8    1.11s
 12   -37.56417144360   +   -4.73       -2.61    2.1    872ms
 13   -37.56442667169       -3.59       -2.87    1.6    1.09s
 14   -37.56449186270       -4.19       -3.03    2.8    844ms
 15   -37

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.